In [1]:
import sys 
sys.path.insert(0, "../../")

from src.neo4j_functions import Neo4jConnection
from src import ReaderMetrics

import json
import requests
from langchain_huggingface import HuggingFaceEmbeddings
import os
from typing import List, Dict, Tuple
from time import sleep, time
from tqdm import tqdm
import numpy as np
import gc
import re
import torch
from collections import Counter
import chromadb
from scipy.spatial import distance

from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

####

EVAL_DATADIR = '../../data/qa_eval'

####

LLM_URL = "https://985f-109-252-76-222.ngrok-free.app/llama" 

####

LOG_DIR = "./logs"

LOG_TMPDATA_DIRNAME = "tmp"
LOG_SCORES_DIRNAME = 'scores'
LOG_STATSDATA_DIRNAME = "stats"

SAVE_STAGE1_LOGDIR = f'{LOG_DIR}/stage1'
SAVE_STAGE2_LOGDIR = f'{LOG_DIR}/final'

LOG_META_FILENMAE = "metadata.json"
LOG_AVGSCORES_FILENAME = 'avgscores.json'

##### Utils

In [2]:
def generate(prompt: str) -> str:
    flag = True
    while flag:    
        try:
            response = requests.post(LLM_URL, params = {"prompt": prompt})
            resp = response.json()["response"]
            flag = False
        except (requests.ConnectionError, requests.ReadTimeout) as e:
            print("Connection error: ", str(e))
            sleep(1)

    return resp

def check_create_dir(new_dir_path: str):
    if os.path.exists(new_dir_path):
        print("Директория существует")
        raise ValueError
    else:
        os.mkdir(new_dir_path)

def save_json(data: Dict[str, object], save_path: str):
    dump = json.dumps(data, ensure_ascii=False, indent=1)
    with open(save_path, 'w', encoding='utf-8') as fd:
        fd.write(dump)

def load_json(load_path: str) -> Dict[str,object]:
    with open(load_path, 'r', encoding='utf-8') as fd:
        data = json.loads(fd.read())
    return data

def round5(number: float) -> float:
    return round(number, 5)

Stage 1. Query llm-classification

In [3]:
PROMPT_QUERIES_LABLE_TEMPLATE = '''You are an expert system and your task is to determe the class of the request in a given question. There is only eight possible request classes to choose from:
- label: "[1]", class description: compare information about the properties of two objects according to a criterion;
- label: "[2]", class description: summarize information about the property of an object from all subjects according to a criterion;
- label: "[3]", class description: summarize information from a subject about an object according to a given criterion;
- label: "[4]", class description: based on information about an object with given properties from a subject with given properties, draw a conclusion according to a criterion;
- label: "[5]", class description: ... ;
- label: "[6]", class description: find overlapping information on properties of objects;
- label: "[7]", class description: determine the most similar object from the list for a given object according to a given criterion based on their properties;
- labels: "[8]", class description: draw a conclusion based on information in relation to an object with given properties from subjects with given properties.

If given question does not belong to any of described classes above return label "[0]". Do not return anything except the predicted label for the given question.

==Examples:
- Question: Which device is better in battery life: iPhone11 Pro Max or Xiaomi 11?
  Label: [1]
- Question: The majority of speakers have positive, neutral or negative sentiment about connection of Apple?
  Label: [2]
- Question: What Jessica's opinion (positive, negative or neutral) about signal of Apple was dominant during using Apple?
  Label: [3]
- Question: Kayla has positive, negative or neutral opinion about video of 10PRO on 25.11.2020?
  Label: [4]
- Question: What opinion (positive, negative or neutral) about scheduling of IQOO9 was last during Zachary's experience of IQOO9?
  Label: [5]
- Question: Do Jane and Jonathan have any common devices (which Jane and Jonathan both use)? If so, list common devices. Otherwise, answer 'No'.
  Label: [6]
- Question: Whose opinions from Freda and Bruce about devices are most similar to Kayla's?
  Label: [7]
- Question: Which people have negative opinion about video of 10PRO on 25.11.2020?
  Label: [8]

==Your Task:
Question: {q}
Label: '''

REGEXP_PATTERN = re.compile(r'\[\d\]')

S1_META = {
    'SAVE_LABELS_VERSION': "v1",
    'CLASSIFY_PROMPT_TEMPLATE': PROMPT_QUERIES_LABLE_TEMPLATE,
    'REGEXP_PATTERN': REGEXP_PATTERN,
    'QUESTION_LABEL_MAP': {
        'other_questions': '0',
        'compare_questions.json': '1', 'compare_sentiment_synonims.json': '2',
        'compare_sentiment.json': '2', 'dominant_opinion.json': '3',
        'device_sentiment.json': '4', 'last_opinion.json': '5',
        'same_device.json': '6', 'same_manufacturer.json': '6',
        'similar_device_opinions.json': '7', 'similar_manuf_opinions.json': '7',
        'which_people_about_device_synonims.json': '8', 'which_people_about_device.json': '8'}
}

In [ ]:
save_log_dir = f"{SAVE_STAGE1_LOGDIR}/{S1_META['SAVE_LABELS_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_statsdata_dir = f"{save_log_dir}/{LOG_STATSDATA_DIRNAME}"

save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"

In [ ]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)
check_create_dir(save_log_statsdata_dir)

In [ ]:
def generate_questions_label(qa_file, save_log_statsdata_dir, save_log_tmpdata_dir):
    print(qa_file)
    questions_data = load_json(f"{EVAL_DATADIR}/{qa_file}")

    generated_labels = []
    process = tqdm(questions_data)
    for q_item in process:
        
        format_input = S1_META['CLASSIFY_PROMPT_TEMPLATE'].format(q=q_item['question'])
        raw_answer = generate(prompt=format_input)

        result = re.findall(S1_META['REGEXP_PATTERN'], raw_answer)
        label = None if result is None else int(result[0][1])
        generated_labels.append({"generated_label": label})

    save_json(generated_labels, f"{save_log_tmpdata_dir}/{qa_file}")

    stats = dict(Counter(generated_labels))
    save_json(stats, f"{save_log_statsdata_dir}/{qa_file}")

In [ ]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files:
    generate_questions_label(qa_file, save_log_statsdata_dir, save_log_tmpdata_dir)
    gc.collect()

save_json(S1_META, save_log_metafile)

Stage final. Quality Measuring

In [ ]:
SF_META = {
    'LOAD_LABELS_VERSION': "v1",
    'SAVE_SCORES_VERSION': "v1"
}

In [ ]:
save_log_dir = f"{SAVE_STAGE2_LOGDIR}/{SF_META['SAVE_SCORES_VERSION']}" 
save_log_scores_dir = f"{save_log_dir}/{LOG_SCORES_DIRNAME}"

load_log_tmpdata_dir = f"{SAVE_STAGE1_LOGDIR}/{SF_META['LOAD_LABELS_VERSION']}/{LOG_TMPDATA_DIRNAME}"
load_log_metafile = f"{SAVE_STAGE1_LOGDIR}/{SF_META['LOAD_LABELS_VERSION']}/{LOG_META_FILENMAE}"

save_log_scoresfile = f"{save_log_dir}/{LOG_AVGSCORES_FILENAME}"
save_log_metafile = f"{save_log_dir}/{LOG_AVGSCORES_FILENAME}"

In [ ]:
check_create_dir(save_log_dir)
check_create_dir(save_log_statsdata_dir)
check_create_dir(save_log_scores_dir)

In [ ]:
def compute_scores(y_true, y_pred):
    return {
        'f1_weighted': round5(f1_score(y_true, y_pred, average='weighted')),
        'recall': round5(recall_score(y_true, y_pred)),
        'precision': round5(precision_score(y_true, y_pred)),
        'accuracy': round5(accuracy_score(y_true, y_pred))
    }

def measure_cls_task_quality(qa_file, qafile_label_map, 
                             save_log_scores_dir, load_log_tmpdata_dir):
    print(qa_file)
    labels_data = load_json(f"{load_log_tmpdata_dir}/{qa_file}")

    pred_labels = list(map(lambda v: v['generated_label'], labels_data))
    true_labels = [qafile_label_map[qa_file]] * len(pred_labels) 

    stat = compute_scores(true_labels, pred_labels)
    save_json(stat, f"{save_log_scores_dir}/{qa_file}")

    return true_labels, pred_labels

In [ ]:
qa_files = os.listdir(EVAL_DATADIR)
qafile_label_map = load_json(load_log_metafile)['QUESTION_LABEL_MAP']

pred_labels, true_labels = [], []
for qa_file in qa_files:
    tmp_t_labels, tmp_p_labels = measure_cls_task_quality(qa_file, qafile_label_map, 
                             save_log_scores_dir, load_log_tmpdata_dir)
   
    true_labels += tmp_t_labels
    pred_labels += tmp_p_labels
    gc.collect()

save_json(compute_scores(true_labels, pred_labels), 
          save_log_scoresfile)
save_json(SF_META, save_log_metafile)